In [1]:
import os
from pathlib import Path
from operator import itemgetter
import pickle
import pydicom
from time import time
import pandas
import traceback

import matplotlib.pyplot as plt
import numpy as np
import cv2
from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection, Point, MultiPoint
from shapely.affinity import translate

from catch_converter.parse_contours import parse_cvi42ws
from LazyLuna.Mini_LL import *
from LazyLuna.utils import *
from LazyLuna.Figures import *
from LazyLuna.Views import *

In [2]:
# basepaths
bp        = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil'
bp_annos  = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/AI'
bp_cases  = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/Cases'
bp_imgs   = '/Users/dietrichhadler/Desktop/Daten/Atria_Hadil/Imgs'

bp        = '/Users/dietrichhadler/Desktop/Daten/Atria_Intra'
bp_annos  = '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/HaTwo'
bp_cases  = '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Cases'
bp_imgs   = '/Users/dietrichhadler/Desktop/Daten/Atria_Intra/Imgs'

In [ ]:
parse_cvi42ws(bp_annos, bp_annos, process=True, debug=False)

In [ ]:
lax_cine_view = LAX_CINE_View()
cases = []

# get the paths of imgs and annotations via SeriesInstanceUIDs
paths = get_imgs_and_annotation_paths(bp_imgs, bp_annos)
print(paths)
print(len(paths))
for imgp, annop in paths:
    #if not '1.3.12.2.1107.5.2.43.167071.30000021082405410237400000007' in annop: continue
    try:
        print(imgp, annop)
        st = time()
        case = Case(imgp, annop, os.path.basename(imgp), os.path.basename(bp_annos))
        case = lax_cine_view.initialize_case(case)
        case.store(bp_cases)
        cases.append(case)
        print('Case customization took: ', time()-st, 'Case: ', case.reader_name, case.case_name)
    except Exception as e:
        print('Failed at: ', imgp, annop, ' , Exception: ', e)
        print(traceback.format_exc())
    print()

In [3]:
cases = [pickle.load(open(os.path.join(bp_cases, p), 'rb')) for p in os.listdir(bp_cases)]
view  = LAX_CINE_View()
#cases = [view.customize_case(c) for c in cases if c.reader_name=='AI']
cases_tmp = []
for c in cases:
    try:
        cases_tmp.append(view.customize_case(c))
    except Exception as e: 
        print('Error in : ', c.case_name)
        print(traceback.print_exc())
        continue
cases = cases_tmp
print('Nr Cases: ', len(cases))

Nr Cases:  58


In [4]:
cases1 = sorted([c for c in cases if c.reader_name=='HaOne'], key=lambda c:c.case_name)
cases2 = sorted([c for c in cases if c.reader_name=='HaTwo'], key=lambda c:c.case_name)
#cases1 = [cases1[0]]+cases1[2:]
for i in range(1):
    print(cases1[i].case_name, cases2[i].case_name)
print(len(cases1))
print(len(cases2))
ccs = [Case_Comparison(c1,c2) for c1,c2 in zip(cases1,cases2)]
print(len(ccs))

Atria_COV_30_ Atria_COV_30_
29
29
29


In [7]:
rows = []
for cc in ccs:
    c1, c2 = cc.case1, cc.case2
    for cr1, cr2 in zip(c1.crs, c2.crs):
        rows.append([cr1.name, cr1.get_val(string=True), cr2.get_val(string=True), cr1.get_val_diff(cr2, string=True)])
from pprint import pprint
pprint(rows)

[['4CV_RAESAREA', '9.79', '9.55', '0.24'],
 ['4CV_RAEDAREA', '16.78', '16.70', '0.08'],
 ['4CV_RAESV', '24.06', '23.20', '0.86'],
 ['4CV_RAEDV', '51.39', '51.22', '0.16'],
 ['4CV_LAESAREA', '10.85', '10.67', '0.18'],
 ['4CV_LAEDAREA', '18.03', '19.64', '-1.62'],
 ['4CV_LAESV', '20.84', '19.65', '1.19'],
 ['4CV_LAEDV', '48.72', '56.42', '-7.70'],
 ['2CV_LAESAREA', '6.83', '6.76', '0.06'],
 ['2CV_LAEDAREA', '11.71', '11.87', '-0.16'],
 ['2CV_LAESV', '12.19', '11.63', '0.55'],
 ['2CV_LAEDV', '25.61', '26.75', '-1.14'],
 ['BIPLANE_LAESV', '19.36', '18.34', '1.01'],
 ['BIPLANE_LAEDV', '39.43', '44.29', '-4.86'],
 ['LAESP_4CV', '0', '0', '0.00'],
 ['LAEDP_4CV', '12', '13', '1.00'],
 ['LAESP_2CV', '0', '0', '0.00'],
 ['LAEDP_2CV', '12', '13', '1.00'],
 ['RAESP_4CV', '0', '0', '0.00'],
 ['RAEDP_4CV', '12', '12', '0.00'],
 ['4CV_RAESAREA', '12.55', '12.07', '0.48'],
 ['4CV_RAEDAREA', '20.91', '20.74', '0.17'],
 ['4CV_RAESV', '37.24', '36.79', '0.44'],
 ['4CV_RAEDV', '69.58', '68.00', '1.58'],
 

In [22]:
def confidence_interval(data):
    return 1.96*np.std(data)/np.sqrt(len(data))

rows = []
for i in range(len(ccs[0].case1.crs)):
    cr_diffs = []
    for cc in ccs:
        c1,  c2  = cc.case1, cc.case2
        cr1, cr2 = c1.crs[i], c2.crs[i]
        #print(c1.case_name, cr1.name, cr1.get_val_diff(cr2))
        cr_diffs.append(cr1.get_val_diff(cr2))
    print(ccs[0].case1.crs[i].name, np.mean(cr_diffs), 1.96*np.std(cr_diffs))
    rows.append([ccs[0].case1.crs[i].name, np.mean(cr_diffs), 1.96*np.std(cr_diffs)])
df = DataFrame(rows, columns=['CR', 'mean', 'conf interval'])
pandas.DataFrame.to_csv(df, '/Users/dietrichhadler/Desktop/clinres.csv', sep=',', decimal='.')

#print(mean_confidence_interval(cr_diffs, 0.95))
#print(confidence_interval(cr_diffs))
#print(np.mean(cr_diffs), np.std(cr_diffs))

4CV_RAESAREA 0.26035880331438493 1.7332566443207216
4CV_RAEDAREA 0.000787742898914088 1.0314969479192224
4CV_RAESV 0.8608627299973773 7.756074616385633
4CV_RAEDV 0.7044064138489077 5.837630970295634
4CV_LAESAREA 0.14582375376557433 1.5176990531757624
4CV_LAEDAREA -0.1999275041212281 2.0823544775978347
4CV_LAESV 1.365296777668458 6.528319680929818
4CV_LAEDV -0.2840917534323571 11.337020708546417
2CV_LAESAREA 0.2664597734393558 1.8144445637243036
2CV_LAEDAREA -0.07434140637872164 1.9788054383447418
2CV_LAESV 1.487658138485679 8.353592609709446
2CV_LAEDV 0.193647337886374 11.839119142020959
BIPLANE_LAESV 1.0836939934058107 6.577721812605698
BIPLANE_LAEDV -0.2743910231159459 10.740393430897534
LAESP_4CV 0.20689655172413793 0.7939581601366424
LAEDP_4CV 0.4827586206896552 1.3244137643738012
LAESP_2CV 0.2413793103448276 0.8387227705566234
LAEDP_2CV 0.4827586206896552 1.3244137643738012
RAESP_4CV 0.2413793103448276 0.9840700264130907
RAEDP_4CV 0.3793103448275862 1.081379310344827


In [21]:
n = 10
cr_diffs = []
for cc in ccs:
    c1,  c2  = cc.case1, cc.case2
    cr1, cr2 = c1.crs[n], c2.crs[n]
    #print(c1.case_name, cr1.name, cr1.get_val_diff(cr2))
    cr_diffs.append(cr1.get_val_diff(cr2))
print(ccs[0].case1.crs[n].name, np.mean(cr_diffs), np.std(cr_diffs))

2CV_LAESV 1.487658138485679 4.262037045770126


In [ ]:
ccs = ccs[3:5]
for cc in ccs:
    cc_table = LAX_CC_Metrics_Table()
    cc_table.calculate(cc)
    print(cc.case1.case_name)
    for cname in ['la', 'ra']: 
        cc_table.present_contour_df(cname, pretty=False)
        display(cc_table.df)

In [ ]:
for cc in ccs:
    print(cc.case1.case_name)
    for cat in cc.case1.categories: print(cat.name, cat.phase, end=', ')
    print()
    for cat in cc.case2.categories: print(cat.name, cat.phase, end=', ')
    print()
    for p in range(cc.case1.categories[0].nr_phases):
        anno = cc.case1.categories[0].get_anno(0,p)
        if len(anno.available_contour_names())==0: continue
        print(p, anno.available_contour_names())
    #break

In [ ]:
#ccs = [ccs[3]]
#print(ccs[0].case1.case_name)
def visualize(ccs):
    cases1   = [cc.case1 for cc in ccs]
    cases2   = [cc.case2 for cc in ccs]
    rows, columns   = 4, 2
    fig, axes = plt.subplots(rows, columns, figsize=(columns*11.0,rows*5.0))
    #self.set_size_inches(w=columns*11.0, h=(rows*6.0))
    #axes = self.subplots(rows, columns)
    custom_palette  = sns.color_palette("Blues")
    custom_palette2 = sns.color_palette("Purples")
    swarm_palette   = sns.color_palette(["#061C36", "#061C36"])
    
    cr_cols = ['case_name']+[cr.name+'_avg' for cr in ccs[0].case1.
                             crs]+[cr.name+'_diff' for cr in ccs[0].case1.crs]
    cr_rows = []
    for cc in ccs:
        row = [cc.case1.case_name]
        for cr1, cr2 in zip(cc.case1.crs, cc.case2.crs):
            row.append((cr1.get_val() + cr2.get_val())/ 2.0)
        for cr1, cr2 in zip(cc.case1.crs, cc.case2.crs):
            row.append(cr1.get_val_diff(cr2))
        cr_rows.append(row)
    cr_table = DataFrame(cr_rows, columns=cr_cols)
    
    display(cr_table)
    x_name, y_name = '4CV_RAESAREA_avg', '4CV_RAESAREA_diff'
    sns.scatterplot(ax=axes[0][0], x=x_name, y=y_name,
                    data=cr_table, markers='o', palette=swarm_palette, 
                    size=np.abs(cr_table[y_name]), s=10, legend=False)
    mean = cr_table[y_name].mean()
    std = cr_table[y_name].std()
    axes[0][0].axhline(mean, ls="-", c=".2")
    axes[0][0].axhline(mean+1.96*std, ls=":", c=".2")
    axes[0][0].axhline(mean-1.96*std, ls=":", c=".2")
    
    x_name, y_name = '4CV_RAEDAREA_avg', '4CV_RAEDAREA_diff'
    sns.scatterplot(ax=axes[0][1], x=x_name, y=y_name,
                    data=cr_table, markers='o', palette=swarm_palette, 
                    size=np.abs(cr_table[y_name]), s=10, legend=False)
    mean = cr_table[y_name].mean()
    std = cr_table[y_name].std()
    axes[0][1].axhline(mean, ls="-", c=".2")
    axes[0][1].axhline(mean+1.96*std, ls=":", c=".2")
    axes[0][1].axhline(mean-1.96*std, ls=":", c=".2")
    
    x_name, y_name = '4CV_LAESAREA_avg', '4CV_LAESAREA_diff'
    sns.scatterplot(ax=axes[1][0], x=x_name, y=y_name,
                    data=cr_table, markers='o', palette=swarm_palette, 
                    size=np.abs(cr_table[y_name]), s=10, legend=False)
    mean = cr_table[y_name].mean()
    std = cr_table[y_name].std()
    axes[1][0].axhline(mean, ls="-", c=".2")
    axes[1][0].axhline(mean+1.96*std, ls=":", c=".2")
    axes[1][0].axhline(mean-1.96*std, ls=":", c=".2")
    
    x_name, y_name = '4CV_LAEDAREA_avg', '4CV_LAEDAREA_diff'
    sns.scatterplot(ax=axes[1][1], x=x_name, y=y_name,
                    data=cr_table, markers='o', palette=swarm_palette, 
                    size=np.abs(cr_table[y_name]), s=10, legend=False)
    mean = cr_table[y_name].mean()
    std = cr_table[y_name].std()
    axes[1][1].axhline(mean, ls="-", c=".2")
    axes[1][1].axhline(mean+1.96*std, ls=":", c=".2")
    axes[1][1].axhline(mean-1.96*std, ls=":", c=".2")
    
    x_name, y_name = '2CV_LAESAREA_avg', '2CV_LAESAREA_diff'
    sns.scatterplot(ax=axes[2][0], x=x_name, y=y_name,
                    data=cr_table, markers='o', palette=swarm_palette, 
                    size=np.abs(cr_table[y_name]), s=10, legend=False)
    mean = cr_table[y_name].mean()
    std = cr_table[y_name].std()
    axes[2][0].axhline(mean, ls="-", c=".2")
    axes[2][0].axhline(mean+1.96*std, ls=":", c=".2")
    axes[2][0].axhline(mean-1.96*std, ls=":", c=".2")
    
    x_name, y_name = '2CV_LAEDAREA_avg', '2CV_LAEDAREA_diff'
    sns.scatterplot(ax=axes[2][1], x=x_name, y=y_name,
                    data=cr_table, markers='o', palette=swarm_palette, 
                    size=np.abs(cr_table[y_name]), s=10, legend=False)
    mean = cr_table[y_name].mean()
    std = cr_table[y_name].std()
    axes[2][1].axhline(mean, ls="-", c=".2")
    axes[2][1].axhline(mean+1.96*std, ls=":", c=".2")
    axes[2][1].axhline(mean-1.96*std, ls=":", c=".2")
    
    m_table = LAX_CCs_MetricsTable()
    m_table.calculate(ccs, LAX_CINE_View())
    display(m_table.df)
    
    ra_vals = m_table.df['ra LAX 4CV RAES DSC'].to_list() + m_table.df['ra LAX 4CV RAED DSC'].to_list()
    la4vals = m_table.df['la LAX 4CV LAES DSC'].to_list() + m_table.df['la LAX 4CV LAED DSC'].to_list()
    la2vals = m_table.df['la LAX 2CV LAES DSC'].to_list() + m_table.df['la LAX 2CV LAED DSC'].to_list()
    
    dicebp = sns.boxplot(ax=axes[3][0], data=[ra_vals,la4vals,la2vals], width=0.4)
    sns.swarmplot(ax=axes[3][0], data=[ra_vals,la4vals,la2vals], 
                  palette=swarm_palette, dodge=True)
    axes[3][0].set_xticklabels(['RA 4CV', 'LA 4CV', 'LA 2CV'])
    axes[3][0].set_ylabel('DSC [%]')
    
    for i, boxplot in enumerate(dicebp.artists):
        if i==0: boxplot.set_facecolor(custom_palette [i])
        else:    boxplot.set_facecolor(custom_palette2[i])
    
    ra_vals = m_table.df['ra LAX 4CV RAES HD'].to_list() + m_table.df['ra LAX 4CV RAED HD'].to_list()
    la4vals = m_table.df['la LAX 4CV LAES HD'].to_list() + m_table.df['la LAX 4CV LAED HD'].to_list()
    la2vals = m_table.df['la LAX 2CV LAES HD'].to_list() + m_table.df['la LAX 2CV LAED HD'].to_list()
    
    hd_bp = sns.boxplot(ax=axes[3][1], data=[ra_vals,la4vals,la2vals], width=0.4)
    sns.swarmplot(ax=axes[3][1], data=[ra_vals,la4vals,la2vals], 
                  palette=swarm_palette, dodge=True)
    axes[3][1].set_xticklabels(['RA 4CV', 'LA 4CV', 'LA 2CV'])
    axes[3][1].set_ylabel('Hausdorff Distance [mm]')
    
    for i, boxplot in enumerate(hd_bp.artists):
        if i==0: boxplot.set_facecolor(custom_palette [i])
        else:    boxplot.set_facecolor(custom_palette2[i])
    
    sns.despine()
    #self.subplots_adjust(left=0.075, bottom=0.05, right=0.95, top=0.95, wspace=0.15, hspace=0.25)
    fig.subplots_adjust(left=0.075, bottom=0.05, right=0.95, top=0.95, wspace=0.15, hspace=0.25)
    
visualize(ccs)

In [ ]:
print(ccs[0].case1.case_name)

for cat in ccs[0].case1.categories:
    print(cat.phase)


print(ccs[0].case1.categories)
print(ccs[0].case2.categories)

In [ ]:
case = [c for c in cases if '444' in c.case_name][0]
print('Case: ', case.case_name)

for cr in case.crs:
    print(cr.name, cr.get_val(string=True))

"""
cat = case.categories[0]
anno = cat.get_anno(0, 0)

print(anno.anno)

for p in range(cat.nr_phases):
    anno = cat.get_anno(0, p)
    print(p, anno.available_contour_names())
    print(p, anno.available_point_names())
    print(anno.length_LA())
    print(anno.length_RA())
#print(case.categories)
#for cr in c.crs:
#    print(cr.name, cr.get_val())
"""

In [ ]:
case = cases[0]
cats = case.categories
#print([c.name for c in cats if 'LAX 4CV' in c.name])
cat = [c for c in cats if 'LAX 4CV LAES' in c.name][0]
print(cat)
print(cat.phase)
for p in range(cat.nr_phases):
    print(cat.get_anno(0,p).anno.keys())
    if len(cat.get_anno(0,p).anno.keys())==0: continue
    img  = cat.get_img(0,p)
    anno = cat.get_anno(0,p)
    fig, ax = plt.subplots(1,1,figsize=(5,5))
    ax.imshow(img, cmap='gray')
    anno.plot_all_contour_outlines(ax)
    plt.show()
    print()
    #break